In [3]:
import cv2
from deepface import DeepFace 
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
import numpy as np
import time
import psycopg2

# duygu dictionary'sinin tanımlanması -> ilk değerleri sıfır
dominantDict = {
  "happy": 0,
  "neutral": 0,
  "sad": 0
}

# analiz edilen duyguların sayısal değerlerini tutmak için value'lar.
happyValue = 0;
neutralValue = 0;
sadValue = 0;

cap = cv2.VideoCapture(1)

# check if the webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam.")
    
# The duration in seconds of the video captured
capture_duration = 16 # n sn için n+1 tanımlanmalı
start_time = time.time()

while( int(time.time() - start_time) < capture_duration ):
    ret,frame = cap.read() # one image from a video
    result = DeepFace.analyze(frame,  actions=['emotion'], enforce_detection=False)
    lastValue = result.get('emotion', {}).get(result['dominant_emotion']) # duyguların sayısal değerleri için variable
    
    # eğer detect edilen baskın duygu değeri happy ise dictionary'de happy key'ine sahip value 1 arttırılır.
    # böylece belirtile sürede kaç kere happy detect edildiğini tutabiliriz.
    # happy'nin ortalamasını almak için önce tüm happy'lerin sayısal değerleri toplanmalı. happyValue ile sayısal happy 
    # değerleri toplanıyor.
    # (aynı açıklamalar neutral ve sad için de geçerli)
    if str(result['dominant_emotion']) == "happy":
        dominantDict["happy"] = dominantDict["happy"] +1
        happyValue = happyValue + result.get('emotion', {}).get(result['dominant_emotion'])

        # kaç kere happy analiz edildiğini yazdırır.
        print("number of happy:", dominantDict["happy"])
        # duygunun toplam sayısal değerini yazdırır.
        print("total happy value:", happyValue)
        print("last happy value:", lastValue)
    
    elif str(result['dominant_emotion']) == "neutral":
        dominantDict["neutral"] = dominantDict["neutral"] +1
        neutralValue = neutralValue + result.get('emotion', {}).get(result['dominant_emotion'])
        
        print("number of neutral:", dominantDict["neutral"])
        print("total neutral value:", neutralValue)
        print("last neutral value:", lastValue)
        
    else:
        dominantDict["sad"] = dominantDict["sad"] +1
        sadValue = sadValue + result.get('emotion', {}).get(result['dominant_emotion'])
        print("number of sad:", dominantDict["sad"])
        print( "total sad value:", sadValue)
        print("last sad value:", lastValue)
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #print (faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    
    #draw a rectangle around the faces
    for(x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
# user putText() method for inserting text on video -> duygu için
    cv2.putText(frame,
           result['dominant_emotion'],
           (0, 50), font, 2,   #(0, 50) -> yazının konumu
           (0, 0, 255), 2,  # color = red
           cv2.LINE_4)
    
# user putText() method for inserting text on video -> sayısal değer için
    cv2.putText(frame,
           str(np.float32(lastValue)),
           (0, 100), font, 1,
           (0, 0, 255), 1,  # color = red
           cv2.LINE_4)
    
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# tutulan duygu değerlerinin karşılaştırılarak en yükseğinin bulunması  

finalResutReal = 0
finalResultEmotion =""

if np.float32(happyValue) > np.float32(neutralValue) and np.float32(happyValue) > np.float32(sadValue):
    print("dominant emotion: happy -> average: ", np.float32(happyValue) / np.float32(dominantDict["happy"]))
    # toplam happy sayısal değeri happy çıkma sayısına bölerek ortalama buluyoruz.
    finalResultReal = np.float32(happyValue) / np.float32(dominantDict["happy"])
    finalResultEmotion = "mutlu"
          
elif np.float32(neutralValue) > np.float32(happyValue) and np.float32(neutralValue) > np.float32(sadValue):
    print("dominant emotion: neutral-> average: ", np.float32(neutralValue) / np.float32(dominantDict["neutral"]))
    finalResultReal = np.float32(neutralValue) / np.float32(dominantDict["neutral"])
    finalResultEmotion = "nötr"
    
elif np.float32(sadValue) > np.float32(happyValue) and np.float32(sadValue) > np.float32(neutralValue):
    print("dominant emotion: sad-> average: ", np.float32(sadValue) / np.float32(dominantDict["sad"]))
    finalResultReal = np.float32(sadValue) / np.float32(dominantDict["sad"])
    finalResultEmotion = "üzgün"
    

cap.release()
cv2.destroyAllWindows()

print("\n",dominantDict) # duygular kaç kere ölçülmüş görmek için dict yazdırdık.
print("\n toplam mutluluk (sayısal değeri):", happyValue)
print("\n toplam nötrlük (sayısal değeri):", neutralValue)
print("\n toplam üzgünlük (sayısal değeri):", sadValue)

number of sad: 1
total sad value: 99.98981952667236
last sad value: 99.98981952667236
number of sad: 2
total sad value: 199.96845722071157
last sad value: 99.97863769403921
number of sad: 3
total sad value: 242.70154535643087
last sad value: 42.7330881357193
number of sad: 4
total sad value: 285.43463349215017
last sad value: 42.7330881357193
number of sad: 5
total sad value: 385.41805743945014
last sad value: 99.98342394729995
number of sad: 6
total sad value: 485.4132056213654
last sad value: 99.99514818191528
number of sad: 7
total sad value: 585.3027463068354
last sad value: 99.88954068547002
number of sad: 8
total sad value: 666.8824006177565
last sad value: 81.57965431092109
number of sad: 9
total sad value: 725.459020242848
last sad value: 58.57661962509155
number of sad: 10
total sad value: 788.8248297590623
last sad value: 63.36580951621419
number of sad: 11
total sad value: 852.1906392752765
last sad value: 63.36580951621419
number of neutral: 1
total neutral value: 88.601672

number of happy: 43
total happy value: 4029.6658003386833
last happy value: 92.92545276435928
number of happy: 44
total happy value: 4116.463456541279
last happy value: 86.79765620259569
number of happy: 45
total happy value: 4198.0576077997475
last happy value: 81.59415125846863
number of happy: 46
total happy value: 4282.285586982945
last happy value: 84.22797918319702
number of happy: 47
total happy value: 4372.490111392985
last happy value: 90.20452441004065
number of neutral: 20
total neutral value: 1582.252708657419
last neutral value: 51.72634720802307
number of happy: 48
total happy value: 4436.894938749324
last happy value: 64.4048273563385
number of happy: 49
total happy value: 4511.556484184606
last happy value: 74.66154543528232
number of happy: 50
total happy value: 4611.5508455852105
last happy value: 99.99436140060425
number of happy: 51
total happy value: 4711.549617729528
last happy value: 99.99877214431763
number of sad: 29
total sad value: 2042.8300427699846
last sad

In [4]:
# veritabanı bağlantısı ve duygu sonucunun ilgili tabloya eklenmesi
from datetime import date
from datetime import datetime
import psycopg2

dbFinalResult = np.float64(finalResultReal) # postgreyle uyumlu olması için float64'e dönüştürdük.
analyseDate = datetime.today().strftime('%Y-%m-%d') # postgresql date formatına dönüştürme

try:
    connection = psycopg2.connect(user="postgres",
                                  password="1234",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres")
    cursor = connection.cursor()

    postgres_insert_query = """ INSERT INTO duygu_veri_analizi (duygu_durumu_deger, duygu_durumu_kategori,birey_id,duygu_durumu_tarih) VALUES (%s,%s,%s,%s)"""
    record_to_insert = (dbFinalResult, finalResultEmotion, '12345678910',analyseDate)
    print("Inserted Values: ",finalResultEmotion ,"-", dbFinalResult)
    cursor.execute(postgres_insert_query, record_to_insert)

    connection.commit()
    count = cursor.rowcount
    print(count, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into table", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Inserted Values:  mutlu - 92.85074615478516
1 Record inserted successfully into table
PostgreSQL connection is closed
